In [1]:
import requests
import os
from datasets import load_dataset
import pandas as pd
import json 

dataset = load_dataset("PatronusAI/financebench")
df = dataset['train'].to_pandas()

In [19]:
finetuning_data = df.rename(columns = {'question':'instruction', 'answer':'output', 'evidence_text': 'input'})

In [22]:
finetuning_data = finetuning_data[['instruction', 'input', 'output']]

In [24]:
finetuning_data

,instruction,input,output
0,What is the FY2018 capital expenditure amount ...,Table of Contents \n3M Company and Subsidiarie...,$1577.00
1,Assume that you are a public equities analyst....,Table of Contents \n3M Company and Subsidiarie...,$8.70
2,Is 3M a capital-intensive business based on FY...,3M Company and Subsidiaries\n Consolidated Sta...,"No, the company is managing its CAPEX and Fixe..."
3,What drove operating margin change as of FY202...,"SG&A, measured as a percent of sales, increase...",Operating Margin for 3M in FY2022 has decrease...
4,"If we exclude the impact of M&A, which segment...",Worldwide Sales Change\nBy Business Segment Or...,The consumer segment shrunk by 0.9% organically.
...,...,...,...
145,Is Verizon a capital intensive business based ...,Consolidated Balance Sheets \nVerizon Communic...,Yes. Verizon's capital intensity ratio was app...
146,Has Verizon increased its debt on balance shee...,"At December 31, Maturities \nInterest \nRates ...",No. Verizon's debt decreased by $229 million.
147,What is FY2018 days payable outstanding (DPO) ...,Walmart Inc.\nConsolidated Statements of Incom...,42.69
148,Based on the information provided primarily in...,Walmart�Inc.\nConsolidated�Statements�of�Incom...,0.2%


In [25]:
text_col = []
for _,row in finetuning_data.iterrows():
    prompt = "You are a financial chatbot trained to answer questions based on the information provided in 10-K documents. Your responses should be directly sourced from the content of these documents. When asked a question, ensure that your answer is explicitly supported by the text in the 10-K filing, and do not include any external information, interpretations, or assumptions not clearly stated in the document. If a question pertains to financial data or analysis that is not explicitly covered in the 10-K filing provided, respond by stating that the information is not available in the document. Your primary focus should be on accuracy, specificity, and adherence to the information in 10-K documents, particularly regarding financial statements, company performance, and market positions.\n"
    instruction = str(row["instruction"])
    input_query = str(row['input'])
    response = str(row['output'])
    
    text = prompt + "### Instruction:\n" +  instruction + "\n### Input: \n" + input_query + "\n### Response: \n " + response
    
    text_col.append(text)

    

In [28]:
finetuning_data.loc[:,'text'] = text_col
finetuning_data

,instruction,input,output,text
0,What is the FY2018 capital expenditure amount ...,Table of Contents \n3M Company and Subsidiarie...,$1577.00,You are a financial chatbot trained to answer ...
1,Assume that you are a public equities analyst....,Table of Contents \n3M Company and Subsidiarie...,$8.70,You are a financial chatbot trained to answer ...
2,Is 3M a capital-intensive business based on FY...,3M Company and Subsidiaries\n Consolidated Sta...,"No, the company is managing its CAPEX and Fixe...",You are a financial chatbot trained to answer ...
3,What drove operating margin change as of FY202...,"SG&A, measured as a percent of sales, increase...",Operating Margin for 3M in FY2022 has decrease...,You are a financial chatbot trained to answer ...
4,"If we exclude the impact of M&A, which segment...",Worldwide Sales Change\nBy Business Segment Or...,The consumer segment shrunk by 0.9% organically.,You are a financial chatbot trained to answer ...
...,...,...,...,...
145,Is Verizon a capital intensive business based ...,Consolidated Balance Sheets \nVerizon Communic...,Yes. Verizon's capital intensity ratio was app...,You are a financial chatbot trained to answer ...
146,Has Verizon increased its debt on balance shee...,"At December 31, Maturities \nInterest \nRates ...",No. Verizon's debt decreased by $229 million.,You are a financial chatbot trained to answer ...
147,What is FY2018 days payable outstanding (DPO) ...,Walmart Inc.\nConsolidated Statements of Incom...,42.69,You are a financial chatbot trained to answer ...
148,Based on the information provided primarily in...,Walmart�Inc.\nConsolidated�Statements�of�Incom...,0.2%,You are a financial chatbot trained to answer ...


In [29]:
finetuning_data.to_csv("train.csv", index = False)

In [30]:
!autotrain llm --train Anote_benchmarking finetuned_model --model meta-llama/Llama-2-7b-hf --data_path . --use-peft --use-4bit --learning_rate 2e-4 --train_batch_size 4 --num_train_epochs 3 --trainer sft 

'autotrain' is not recognized as an internal or external command,
operable program or batch file.


In [31]:
!pip install autotrain-advanced

  Obtaining dependency information for transformers==4.36.1 from https://files.pythonhosted.org/packages/fc/04/0aad491cd98b09236c54ab849863ee85421eeda5138bbf9d33ecc594652b/transformers-4.36.1-py3-none-any.whl.metadata
  Using cached transformers-4.36.1-py3-none-any.whl.metadata (126 kB)
Using cached transformers-4.36.1-py3-none-any.whl (8.3 MB)
  Attempting uninstall: transformers
    Found existing installation: transformers 4.34.0
    Uninstalling transformers-4.34.0:
      Successfully uninstalled transformers-4.34.0


In [32]:
!autotrain setup --update-torch

'autotrain' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:
!autotrain llm --train Anote_benchmarking finetuned_model --model meta-llama/Llama-2-7b-hf --data_path . --use-peft --use-4bit --learning_rate 2e-4 --train_batch_size 4 --num_train_epochs 3 --trainer sft 

In [33]:
!autotrain --help

'autotrain' is not recognized as an internal or external command,
operable program or batch file.


In [34]:
!pip install autotrain

ERROR: Could not find a version that satisfies the requirement autotrain (from versions: none)
ERROR: No matching distribution found for autotrain
